## QM/MM Simulation of Proton transfer in Phenanthrolin

This tutorial provides an example of running up QM/MM MD simulations using ASH. 

Requirements:
* ASH: https://ash.readthedocs.io/en/latest/About.html \\
* xTB: https://xtb-docs.readthedocs.io/en/latest/
* OpenMM: https://openmm.org/
* NGLView and MDTraj for visualisation

We will simulate a simple intramolecular proton of phenothrolin. Below the solvated molecule is visualized using NGLView and MDTraj.

In [ ]:
import nglview as ngl
import mdtraj as md

mol = md.load_pdb('../data/phenanthrolin_solv.pdb')
view = ngl.show_mdtraj(mol)
view.clear_representations()
view.add_licorice(opacity=0.25)
view.add_representation('ball+stick', '1')
view

**First, an ASH fragment is created from the PDB.**

In [ ]:
import ash

mm_path = "../data/phenanthrolin_solv"
frag = ash.Fragment(
    pdbfile=f"{mm_path}.pdb", 
    charge=1, # charge of QM region (total charge is 0)
    mult=1,   # multiplicity
)

**Next, an OpenMMTheory object is created from the ASH fragment using AMBER parameters.**

In [ ]:
mm_theory = ash.OpenMMTheory(
    cluster_fragment=frag,               # ASH Fragment 
    Amberfiles=True,                     # use Amber parameters
    amberprmtopfile=f"{mm_path}.parm7",  # Amber parameter file
    autoconstraints=None,                # Type of automatic constraints to apply to system. Options: 'HBonds', 'AllBonds', 'HAngles'
    constraints=None,                    # List of lists of constraint definitions based on atom indices. 
    rigidwater=False,                    # Whether to automatically apply rigid water constraints for recognized water models
    hydrogenmass=1.0,                    # Hydrogen mass repartioning value. 1.5 is OpenMM and ASH default.
    platform="CPU",                      # OpenMM platform
    numcores=4,                          # number of CPU cores
    printlevel=0,                        # The printlevel 
    periodic=True,                       # Periodic boundary conditions or not.
)

**For the sake of this tutorial we will use xTB as a QM method.**

In [ ]:
qm_theory = ash.xTBTheory(
    xtbmethod="GFN2",    # The xTB method
    runmode="inputfile", # Only inputfile supported for QM/MM
    numcores=4,          # Number of CPU cores
    printlevel=0,        # The printlevel
)

**Finally, the QMMMTheory is created, connecting the QM theory to the OpenMMTheory.**

In [ ]:
qm_atoms = [i for i in range(0, 23)] # QM atoms

qmmm_theory = ash.QMMMTheory(
    qm_theory=qm_theory,        # ASH QM Theory object
    mm_theory=mm_theory,        # ASH MM Theory object (should be OpenMMTheory)
    fragment=frag,              # ASH Fragment
    embedding="Elstat",         # QM/MM embedding type
    qmatoms=qm_atoms,           # The QM atoms (list of atom indices)
    printlevel=0,               # The printlevel
    unusualboundary=False,      # Optional: Boundary-option: overrides ASH from quitting if an unusual QM-MM boundary is found.
    TruncatedPC=False,          # Optional: Truncated Pointcharge Option on or off.
    TruncPCRadius=55.0,         # Optional: Truncated PC option; Radius (Å) for the truncated PC region.
    TruncatedPC_recalc_iter=50, # Optional: Truncated PC option; frequency for recalculating with full PC field.
)

**To check the validity of the QM/MM Theory we perform a singlepoint calculation.**

In [ ]:
result = ash.Singlepoint(theory=qmmm_theory, fragment=frag, Grad=False)

### Molecular Dynamics (MD)

Now that the QM/MM system was successfully created, we will perform an MD simulation using the adaptive-sampling package. 

To control the temperature, a Langevin thermostat will be applied, while the pressure is controlled by a Monte-Carlo Barostat. 

To enhance sampling of the proton transfer, OPES-eABF will be applied to a linear combination of the breaking and forming N-H bonds.

In [ ]:
# Init the MD 
from adaptive_sampling.interface.interfaceASH import AshMD

the_md = AshMD(
    fragment=frag,                             # ASH fragment
    calculator=qmmm_theory,                    # ASH calculator 
    dt=1.0,                                    # Time step in fs
    thermostat=True,                           # Apply Langevin thermostat 
    friction=1.0e-3,                           # Friction for Langevon thermostat
    target_temp=300.0,                         # The target temperature in Kelvin
    barostat=True,                             # Apply Monte-Carlo barostat
    target_pressure=1.0,                       # The target pressure in Bar
    barostat_freq=25,                          # Frequency of updating the barostat
    barostat_reporter='barostat.log',          # Store barostat information 
    seed=42,                                   # The random seed 
)

# Init bias potentials and confinements
from adaptive_sampling.sampling_tools import OPESeABF

# setup the Collective Variable (CV)
cv_type = "lin_comb_dists"       # linear combination of bond distances
cv_def = [
    ['distance', -1.0, [2,22]],  # proton distance to N1
    ['distance', 1.0, [12,22]],  # proton distance to N2
]
cv_min = -2.0                    # Minimum of CV grid
cv_max = 2.0                     # Maximum of CV grid
cv_binwidth = 0.05               # Bin width for CV grid
the_cv = [[cv_type, cv_def, cv_min, cv_max, cv_binwidth], ]

# setup the sampling algorithm
ext_sigma       = 0.05  # coupling width for extended system
ext_mass        = 20.0  # mass of fictitious particles of extended system
opes_kernel_std = 0.2   # Initial standard deviation of kernels for OPES

the_bias = OPESeABF(
    the_md,
    the_cv,
    ext_sigma=ext_sigma,
    ext_mass=ext_mass,
    kernel_std=opes_kernel_std,
    f_conf=100.0,
    output_freq=1000,
)

the_md.calc_init(
    init_momenta="random", 
    biaspots=[the_bias, ], 
    init_temp=300.0
)

**Run the MD.**

Note, that only 1000 MD steps are performed here for demonstration because of the associated computational cost. 
To obtain converged results a much longer simulation is required. 

In [ ]:
the_md.run(
    nsteps=1000,               # number of steps
    out_freq=100,              # frequency of writing MD output 
    dcd_freq=10,               # frequency of writing coordinates to dcd trajectory 
    restart_freq=100,          # frequency of writing restart files
    remove_rotation=False,     # remove center of mass rotation in MD (not needed with periodic boundary conditions)
    prefix=f"Production_01",   # prefix for AshMD output files 
)

## Visualize results. 

**Note that to converge results, a much longer simulation than 1000 MD steps is required.**

In [ ]:
mol = md.load_dcd('Production_01_traj.dcd', top='../data/phenanthrolin_solv.parm7')
view = ngl.show_mdtraj(mol)
view.clear_representations()
view.add_licorice(opacity=0.25)
view.add_representation('ball+stick', '0')
view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cv_traj = np.loadtxt("CV_traj.dat", skiprows=1)
barostat_log = np.loadtxt("barostat.log")

At atmospheric pressure the density of the periodic box should converge roughly to 1.0 g/cm$^3$ due to the barostat. 

Afterwards, the system is equilibrated and the remaning data can be used for production (e.g. to calculate the free energy profile of the proton transfer).

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (fs)', fontsize=20)
ax1.set_ylabel('Volume (nm$^3$)', color=color, fontsize=20)
ax1.plot(barostat_log[:,0], barostat_log[:,1], color=color)
ax1.tick_params(axis='y', labelcolor=color, labelsize=15)
ax1.tick_params(axis='x', labelsize=15)

ax2 = ax1.twinx()  

color = 'tab:blue'
ax2.set_ylabel('Density (g/cm$^3$)', color=color, fontsize=20) 
ax2.plot(barostat_log[:,0], barostat_log[:,2], color=color)
ax2.tick_params(axis='y', labelcolor=color, labelsize=15)

fig.tight_layout() 
plt.show()

Below, the trajectory of the CV is plotted. In longer simulations, as the proton jumps back and forth between both N-atoms because of the OPES-eABF bias, it should diffuse between negative and positive values, indicating that the proton is bound to one or the other N atom.  

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(cv_traj[:,0], cv_traj[:,1], color='tab:red')                # trajectory of the CV
ax1.plot(cv_traj[:,0], cv_traj[:,2], color='tab:blue', linewidth=4)  # trajectory of the extended-system, that is coupled to the CV
ax1.set_xlabel('time (fs)', fontsize=20)
ax1.set_ylabel('CV', fontsize=20)
ax1.tick_params(axis='y', labelsize=15)
ax1.tick_params(axis='x', labelsize=15)

fig.tight_layout() 
plt.show()

If multiple transitions between both minima are observed, the potential of mean force (PMF), i.e., free energy profile, of the transition can be calculated using thermodynamic integration or using the MBAR estimator. Below, the former is used as we are only interested in the PMF and do not aim for general reweighting of the trajectory.

In [ ]:
from adaptive_sampling.processing_tools import thermodynamic_integration as ti
from adaptive_sampling import units

ext_sigma = 0.05                       # Coupling width of the extended system as applied in the simulation
grid      = np.linspace(-1.4, 1.4, 57) # Grid for the CV
bin_width = grid[1]-grid[0]            # Bin width of the grid

grad_pmf = ti.czar(
    grid=grid,                         # CV grid
    cv=cv_traj[:,1],                   # Trajectory of the CV
    la=cv_traj[:,2],                   # Trajectory of the extended system
    sigma=ext_sigma,                   # Coupling width of the extended system
    equil_temp=300.0,                  # Temperature of the simulation
    periodicity=None,                  # Periodicity of the CV
)

pmf, rho = ti.integrate(
    grad_pmf,                          # Gradient of the PMF
    bin_width,                         # Bin width
    equil_temp=300.0,                  # Temperature of the simulation
    method='simpson',                  # Integrate using Simpson's rule
)

If the simulation is convergent, the PMF should be symmetrical and resemble a double well potential.

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(grid, pmf, color='tab:red') 
ax1.set_xlabel(r'CV / A', fontsize=20)
ax1.set_ylabel('A / kcal mol$^{-1}$', fontsize=20)
ax1.tick_params(axis='y', labelsize=15)
ax1.tick_params(axis='x', labelsize=15)

fig.tight_layout() 
plt.show()